In order for algorithms to be stable and not affected by other parts of the application, the algorithms should run in a different thread or even application. They should be as isolated from the rest of the Smart home, especially from its intelligent and communication components, as much as possible. But in this case, how do we monitor and control them?

The communication between algorithm and the externals is performed by using SQLite. We have two classes that incapsulate SQL commands and offer high-level abstraction for them:

* `IStorage` that saves all the data from spaces as unstructured data. We don't create columns for each of the space fields: first, we don't really need it; second, that would bring all sorts of the migration problems when changing these fields. Instead, we `jsonpickle` the states' current values. `jsonpickle` is used instead of `pickle` to allow reading the logs even in the absence of the source code. 
* `IMessenger` is a messaging queue: sender sends a message with various tags, and receiver can get the messages of the specified tags in a time-sorted order, and close them with replies.

`IStorage`, `IMessenger` and their SQL implementations are located in `kaia.infra`.

For `bro`, there are developed units that read and handle the requests to change settings that come from `IMessenger`. Also, by default, `BroAlgorithm` dumps the current values of all the fields in the space to `IStorage`.

Based on this, there are two control systems created. The first is with `gradio` that shows the data from the space as an interactive plot, and allows controlling the algorithm with wegbui elements. The second is with `eaglesong` that allows some of the controlling via `Telegram`.

Let's demonstrate how all this works. We will use the control bot from the previous demo. It's defined in `kaia.bro.sandbox.control_space`

In [1]:
from kaia.bro.sandbox.control_space import ControlSpace
from kaia.bro.core import BroServer
from kaia.infra.comm import Sql

storage = Sql.file().storage()
messenger = Sql.file().messenger()


algorithm = ControlSpace().create_algorithm(0.5)
server = BroServer([algorithm], pause_in_milliseconds=100)
server.run_in_multiprocess(storage, messenger)

<Process name='Process-1' pid=9952 parent=21176 started>

Now the server is running in the background. Let's see the gradio interface:

In [2]:
from kaia.bro.amenities.gradio import GradioClient
from kaia.bro.core import BroAlgorithmPresentation


gradio_client = GradioClient(server.create_client(algorithm, storage, messenger), algorithm.presentation)
interface = gradio_client.generate_interface()
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
